<a href="https://colab.research.google.com/github/analyticswithadam/Python/blob/main/Automate_Voice_of_the_Customer_with_the_YouTube_API_and_Gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# Pull Data from YouTube API

[Playlist - BMW i-5](https://www.youtube.com/watch?v=aOuefVQHVeo&list=PLa5S6UwF22ILpD4WOSM-TbbH-fBxmFbqU)

### Imports

In [ ]:
from googleapiclient.discovery import build
import pandas as pd
from google.colab import files, drive
import getpass
from IPython.display import display, Markdown

## User Input

In [ ]:
api_key = getpass.getpass('Please enter your YouTube API key: ')
playlist_ids = ['PLa5S6UwF22ILpD4WOSM-TbbH-fBxmFbqU']  # Replace with your playlist IDs


Please enter your YouTube API key: ··········


In [ ]:
# Build the YouTube client
youtube = build('youtube', 'v3', developerKey=api_key)

## Get Video IDs for Playlist

In [ ]:
def get_all_video_ids_from_playlists(youtube, playlist_ids):
    all_videos = []  # Initialize a single list to hold all video IDs

    for playlist_id in playlist_ids:
        next_page_token = None

        # Fetch videos from the current playlist
        while True:
            playlist_request = youtube.playlistItems().list(
                part='contentDetails',
                playlistId=playlist_id,
                maxResults=50,
                pageToken=next_page_token)
            playlist_response = playlist_request.execute()

            all_videos += [item['contentDetails']['videoId'] for item in playlist_response['items']]

            next_page_token = playlist_response.get('nextPageToken')

            if next_page_token is None:
                break

    return all_videos

# Fetch all video IDs from the specified playlists
video_ids = get_all_video_ids_from_playlists(youtube, playlist_ids)

# Now you can pass video_ids to the next function
# next_function(video_ids)

In [ ]:
# Function to get replies for a specific comment
def get_replies(youtube, parent_id, video_id):  # Added video_id as an argument
    replies = []
    next_page_token = None

    while True:
        reply_request = youtube.comments().list(
            part="snippet",
            parentId=parent_id,
            textFormat="plainText",
            maxResults=100,
            pageToken=next_page_token
        )
        reply_response = reply_request.execute()

        for item in reply_response['items']:
            comment = item['snippet']
            replies.append({
                'Timestamp': comment['publishedAt'],
                'Username': comment['authorDisplayName'],
                'VideoID': video_id,
                'Comment': comment['textDisplay'],
                'Date': comment['updatedAt'] if 'updatedAt' in comment else comment['publishedAt']
            })

        next_page_token = reply_response.get('nextPageToken')
        if not next_page_token:
            break

    return replies

# Function to get all comments (including replies) for a single video
def get_comments_for_video(youtube, video_id):
    all_comments = []
    next_page_token = None

    while True:
        comment_request = youtube.commentThreads().list(
            part="snippet",
            videoId=video_id,
            pageToken=next_page_token,
            textFormat="plainText",
            maxResults=100
        )
        comment_response = comment_request.execute()

        for item in comment_response['items']:
            top_comment = item['snippet']['topLevelComment']['snippet']
            all_comments.append({
                'Timestamp': top_comment['publishedAt'],
                'Username': top_comment['authorDisplayName'],
                'VideoID': video_id,  # Directly using video_id from function parameter
                'Comment': top_comment['textDisplay'],
                'Date': top_comment['updatedAt'] if 'updatedAt' in top_comment else top_comment['publishedAt']
            })

            # Fetch replies if there are any
            if item['snippet']['totalReplyCount'] > 0:
                all_comments.extend(get_replies(youtube, item['snippet']['topLevelComment']['id'], video_id))

        next_page_token = comment_response.get('nextPageToken')
        if not next_page_token:
            break

    return all_comments

# List to hold all comments from all videos
all_comments = []


for video_id in video_ids:
    video_comments = get_comments_for_video(youtube, video_id)
    all_comments.extend(video_comments)

# Create DataFrame
comments_df = pd.DataFrame(all_comments)





# View Comments

In [ ]:
comments_df.head()

,Timestamp,Username,VideoID,Comment,Date
0,2024-03-26T14:47:32Z,@ThaboMuliwana-vd2cb,aOuefVQHVeo,"Ceo needs to resign, 😵‍💫",2024-03-26T14:47:32Z
1,2024-03-21T18:07:49Z,@pecasrapidas,aOuefVQHVeo,"Come to brazil, please!! 😍😍",2024-03-21T18:07:49Z
2,2024-03-13T21:51:14Z,@mireyarosa,aOuefVQHVeo,😂 ok,2024-03-13T21:51:14Z
3,2024-02-15T00:15:42Z,@thomasklaiber4740,aOuefVQHVeo,"If you need this device, you don't need a BMW.",2024-02-15T00:15:42Z
4,2024-01-20T20:13:52Z,@Raza-pz8vx,aOuefVQHVeo,Bmw need a new design team,2024-01-20T20:13:52Z


In [ ]:
comments_df.Comment

0                            Ceo needs to resign, 😵‍💫
1                         Come to brazil, please!! 😍😍
2                                                😂 ok
3      If you need this device, you don't need a BMW.
4                          Bmw need a new design team
                            ...                      
170            how do you not understand english? lol
171                           Looks great for a TAXI🤣
172                                              nice
173                                       Amezing car
174                                             First
Name: Comment, Length: 175, dtype: object

In [ ]:
# Convert the comments_df dataframe to markdown
markdown_text = comments_df.Comment.to_markdown()

# Prompt Gemini


In [ ]:
!pip install --upgrade google-cloud-aiplatform

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
markdown_text

"|     | Comment                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 |\n|----:|:------------------------------------------------------------------------------------------------------------

In [ ]:
from vertexai.preview.generative_models import GenerativeModel, Part
import vertexai
import vertexai.preview.generative_models as generative_models

prompt = f"""
               Role
              -------
              You are a Social Media Marketing Manager at BMW

              Task
              ------
              You just ran a campaign on YouTube for a new product the BMW i5.
              Which involved 5 videos, I will provide you with the comments received on these YouTube Videos

              Assess these comments for sentiment noting the volume of positive negative and neutral comments.
              Please provide some example comments to back up your points on each section.

              From the analysis suggest three things we should keep doing and three things we should avoid doing in future campaigns

              Comments
              ------
              {markdown_text}



"""

generation_config = {
    "max_output_tokens": 5000,
    "temperature": 0,
    "top_p": 0.9,
}
safety_settings = {
    generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
}

def generate():
  vertexai.init(project="YOUR PROJECT HERE", location="YOUR PROJECT LOCATION HERE")
  model = GenerativeModel("gemini-1.5-pro-preview-0409")
  responses = model.generate_content(
      [prompt],
      generation_config=generation_config,
      safety_settings=safety_settings,
      stream=False
  )
  return responses.text

In [ ]:
display(Markdown(generate()))

## BMW i5 YouTube Campaign Sentiment Analysis 

After reviewing the comments for the BMW i5 YouTube campaign, here's a breakdown of the sentiment:

**Positive Sentiment (approx. 60%):**  A significant portion of the comments express positive sentiment towards the BMW i5, its features, and the overall campaign. 

**Example Comments:**

*   "Oh now this is an ad campaign I can get behind. Rock the new i5, BMW <3 Can't wait to see it in person"
*   "I love my 5 series anyway, but this looks incredible 😮"
*   "The new era of innovation! Well done BMW, that's why you're the best!"
*   "Beautiful work, now the 5 series takes the crown on its category 1000000% ❤"

**Negative Sentiment (approx. 20%):**  Negative comments focus on specific issues like design preferences, feature availability in certain markets (like Australia), past experiences with BMW, comparisons with competitors (Tesla, Mercedes), and concerns about the brand's direction.

**Example Comments:**

*   "Dont trust in  bmw advertise because they not working at all. I buy new a bmw 5 g30 5  years ago  and the advertise was idrive amd 3 d maps but steel today they dont work in my car amd in any bmw car in balkan."
*   "BMW didn't get the memo that car thieves steal cars by accessing the car owner's key Pod with electronic key pod scanners."
*   "Are these features actually going to be available in the Australian cars or are you just going to switch them off like you did with the i4s lane change assist? I know that wasn't about the laws here because my wife's 2021 Merc has it..."
*   "This is only available in USA and Germany for now"

**Neutral Sentiment (approx. 20%):** Neutral comments include expressions of desire for the car, simple statements of fact, or requests for more information.

**Example Comments:**

*   "Come to brazil, please!! 😍😍"
*   "BMW, what are you doing  ?!?!  you forget about your main family trait. you forget about genetics - this is a classic chassis layout! make the classic layout more pronounced, move the front axle forward.  BMW, good luck to you !"
*   "Optional upgrade?"
*   "I am planning on getting an M3 comp, but now I want an i7."

## Recommendations for Future Campaigns

**Keep Doing:**

1.  **Showcase Innovation:** The positive response to the i5's technology suggests a continued focus on highlighting innovative features in future campaigns.
2.  **High-Quality Production:** Many comments praise the campaign's visuals and overall quality. Maintaining this standard will be crucial.
3.  **Engage with Enthusiasts:**  Respond to comments and questions, especially from those expressing interest in the i5 or BMW in general.

**Avoid Doing:**

1.  **Overlooking Regional Concerns:** Address concerns about feature availability in specific markets directly to avoid alienating potential customers.
2.  **Ignoring Criticisms:** While positive sentiment is high, address recurring criticisms (e.g., design language, comparisons with competitors) to mitigate negative perceptions.
3.  **Misrepresenting Features:** Ensure that advertised features are accurate and available, as misleading information can severely damage trust. 
